In [1]:
#Imports
import os
import tensorflow as tf

import sys
sys.path.insert(0, '..')
from utils.federatedAggregation import FederatedAggregation
from utils.reinforcementLearningHelper import *
from utils.federatedLearningHelper import *

In [2]:
#Setup Environments of selected buildings for training, evaluation, and testing

environments, observation_spec, action_spec  = setup_energymanagement_environments(num_buildings=30)

#Check environment setup
print(
    "Batch size:", environments["train"][f"building_1"].batch_size, 
    "/ State Space: {} / Action Space: {}".format(observation_spec.shape[0], action_spec.shape[0]),
    "/ Upper bound: {}".format(round(environments["train"][f"building_1"].action_spec().maximum.item(), 3)),
)

Batch size: 1 / State Space: 22 / Action Space: 1 / Upper bound: 2.3


In [3]:
# Cluster similar buildings (K-Means with DTW)

#Set parameter
num_clusters = 8 # 9, 10, 11, 12

clustered_buildings = prosumption_clustered_buildings(num_clusters)
clustered_buildings

{0: array([7], dtype=int64),
 1: array([16, 19, 24], dtype=int64),
 2: array([ 1, 11, 12, 27], dtype=int64),
 3: array([ 2,  4,  6,  9, 10, 14, 15, 18, 22, 25, 30], dtype=int64),
 4: array([20, 26, 28], dtype=int64),
 5: array([13, 17, 21, 23, 29], dtype=int64),
 6: array([8], dtype=int64),
 7: array([3, 5], dtype=int64)}

In [4]:
# Setup Agent networks
federated_rounds = 5
batch_size = 128
replay_buffer_capacity = 20000 #-> only <18.000 samples per dataset
initial_collect_steps = 2000 #2000
collect_steps_per_iteration = 30 
num_iterations = 10000 #10000
eval_interval = num_iterations-10

In [6]:
# FEDERATED LEARNING - Initalization Round 0

tf.compat.v1.reset_default_graph()
global_step = tf.compat.v1.train.get_or_create_global_step()

#Initalize a global model for each Cluster of similar buildings
for cluster in range(num_clusters):
        
        # 1. Build global agent per cluster
        first_building_in_cluster = clustered_buildings[cluster][0]
        global_ddpg_agent, global_eval_policy, global_collect_policy = initialize_ddpg_agent(
                observation_spec=observation_spec, action_spec=action_spec, global_step=global_step, environments=environments,
                )

        # 2. Initially store weights
        model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster}/FLround{0}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        
        save_ddpg_weights(global_ddpg_agent, model_dir)

In [ ]:
#FEDERATED LEARNING - Model training for multiple Rounds

#For each federated round and cluster
for federated_round  in range(federated_rounds):
    for cluster_number, buildings_in_cluster in clustered_buildings.items():

        #Iterate through the buildings per cluster
        print(f"Cluster {cluster_number}: Buildings {buildings_in_cluster} Federated round ---", federated_round+1, f"/ {federated_rounds}")
        local_storage = {
            "actor_weights": [], "critic_weights": [], "target_actor_weights": [], "target_critic_weights": [],"performance_metrics": []
            }
        
        for building_index in buildings_in_cluster:
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()
            
            #1. Initalize local agent
            local_ddpg_agent, local_eval_policy, local_collect_policy = initialize_ddpg_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster_number}/FLround{federated_round}_c{num_clusters}_AvgAgg")
            local_ddpg_agent = set_weights_to_ddpg_agent(local_ddpg_agent, model_dir)
            
            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            local_iterator, local_collect_driver, local_time_step, local_policy_state = setup_rl_training_pipeline(
                local_ddpg_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                local_collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Train, evaluate agent and store weights
            local_ddpg_agent, local_storage = local_agent_training_and_evaluation(
                local_iterator, local_collect_driver, local_time_step, local_policy_state, global_step, 
                local_ddpg_agent, local_eval_policy, local_storage, building_index, num_iterations, environments, agent_type="ddpg"
                )           

        # Performe Federated aggregation
        average_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["actor_weights"], local_storage["performance_metrics"])
        average_critic_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["critic_weights"], local_storage["performance_metrics"]) 
        average_target_actor_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["target_actor_weights"], local_storage["performance_metrics"]) 
        average_target_critic_weights = FederatedAggregation.federated_weigthed_aggregation(local_storage["target_critic_weights"], local_storage["performance_metrics"])    
        
        #Save federated weights for next round (Round + 1)
        model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster_number}/FLround{federated_round+1}_c{num_clusters}_AvgAgg")
        os.makedirs(model_dir, exist_ok=True)
        save_federated_ddpg_weights(model_dir, average_actor_weights, average_critic_weights, average_target_actor_weights, average_target_critic_weights)

Cluster 0: Buildings [7] Federated round --- 1 / 3
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.
Return:  -3902.5432
Cluster 1: Buildings [16 17 21] Federated round --- 1 / 3
Return:  -8007.1987
Return:  -16120.983
Return:  -10929.29
Cluster 2: Buildings [13 19 20] Federated round --- 1 / 3
Return:  -3111.2246
Return:  -1599.379
Return:  -3679.7766
Cluster 3: Buildings [18] Federated round --- 1 / 3
Return:  -7357.402
Cluster 4: Buildings [ 3  4  9 14 15 22 30] Federated round --- 1 / 3
Return:  -2641.6665
Return:  -4560.692
Return:  -4684.999
Return:  -2392.49
Return:  -2660.7192
Return:  -3788.0754
Return:  -3010.6033
Cluster 5: Buildings [1] Federated round --- 1 / 3
Return:  -4835.6523
Cluster 6: Buildings [6 8] Federated round --- 1 / 3
Return:  -9488.9375
Return:  -12512.76
Cluster 7: Buildings [11] Federated round --- 1 / 3
Return:  -5485.527
Cluster 8: Buildings [ 5 12 23 24 25 26 27 28 29] Federated round --- 1 / 3
Return:  -3432.037

In [ ]:
# LOCAL REFITTING AND EVALUATION

best_federated_round = 3
num_rounds=3
num_test_iterations = 1000

result_df = pd.DataFrame(columns=['Building', 'Total Profit'])

for cluster_number, buildings_in_cluster in clustered_buildings.items():
    for building_index in buildings_in_cluster:
        
        for round in range(num_rounds):
            print("Cluster: ", cluster_number, " - Building: ", building_index, " - round: ", round)
            
            #0. Reset global step
            tf.compat.v1.reset_default_graph()
            global_step = tf.compat.v1.train.get_or_create_global_step()

            #1. Initalize local agent
            tf_ddpg_agent, eval_policy, collect_policy = initialize_ddpg_agent(
                observation_spec = observation_spec, action_spec = action_spec,
                global_step = global_step, environments = environments,
                )
            
            #2. Set global weights of this training round to agent (loads the weights of last training)
            model_dir = os.path.join(os.getcwd(), f"models/ddpg/cluster_{cluster_number}/FLround{best_federated_round}_c{num_clusters}_AvgAgg")
            tf_agent = set_weights_to_ddpg_agent(tf_ddpg_agent, model_dir)

            #3. Prepare training pipeline: Setup iterator, replay buffer, driver
            iterator, collect_driver, time_step, policy_state = setup_rl_training_pipeline(
                tf_ddpg_agent, environments["train"][f"building_{building_index}"], replay_buffer_capacity,
                collect_policy, initial_collect_steps, collect_steps_per_iteration, batch_size
                )
            
            #4. Setup wandb logging
            artifact = initialize_wandb_logging(name=f"Exp_building{building_index}_rd{round}", num_iterations=num_iterations)
            
            #5. Train, evaluate agent and store weights
            result_df, metrics = agent_training_and_evaluation(global_step, num_test_iterations, collect_driver, 
                time_step, policy_state, iterator, tf_ddpg_agent, eval_policy, building_index, result_df, eval_interval, environments)
            
            #6. End and log wandb
            end_and_log_wandb(metrics, artifact)

Cluster:  0  - Building:  7  - round:  0


c:\Users\rs1044\Documents\GitHub\Federated-Reinforcement-Learning-for-Battery-Charging\src\notebooks\..\utils\reinforcementLearningHelper.py:352: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result_df = pd.concat([result_df, pd.DataFrame({'Building': [building_index], 'Total Profit': [wandb.summary["Final Profit"]]})], ignore_index=True)


Building:  7  - Total Profit:  -634.6005570423845
Cluster:  1  - Building:  16  - round:  0
Building:  16  - Total Profit:  -1163.5728369061396
Cluster:  1  - Building:  17  - round:  0
Building:  17  - Total Profit:  -1795.252754543063
Cluster:  1  - Building:  21  - round:  0
Building:  21  - Total Profit:  -1957.3334600872965
Cluster:  2  - Building:  13  - round:  0
Building:  13  - Total Profit:  -532.1620693450454
Cluster:  2  - Building:  19  - round:  0
Building:  19  - Total Profit:  -347.398763576045
Cluster:  2  - Building:  20  - round:  0
Building:  20  - Total Profit:  -1462.7135093825673
Cluster:  3  - Building:  18  - round:  0
Building:  18  - Total Profit:  -1712.4691824865156
Cluster:  4  - Building:  3  - round:  0
Building:  3  - Total Profit:  -310.0125272292778
Cluster:  4  - Building:  4  - round:  0
Building:  4  - Total Profit:  -961.5904493064352
Cluster:  4  - Building:  9  - round:  0
Building:  9  - Total Profit:  -834.8373240463116
Cluster:  4  - Building

In [6]:
result_df

NameError: name 'result_df' is not defined

In [ ]:
# Save results
result_df.rename(columns={'Total Profit': 'Profit'}, inplace=True)
result_df['Setup'] = f'cluster{num_clusters}'
result_df.index.name = 'Building_nr'
result_df.reset_index(inplace=True, drop=True)
os.makedirs('results', exist_ok=True)
#result_df.to_csv(f'results/results_clusters{num_clusters}_df.csv', index=False)
print("Final reuslt - clusters ", num_clusters, ": ", result_df["Profit"].sum())

Final reuslt - clusters  10 :  -33979.25013685246
